In [2]:
import requests
import os
from dotenv import load_dotenv
import time
import pandas as pd
import import_ipynb

In [3]:
import open_weather_coordinates

Stadt: London, Land: GB, Breitengrad: 51.5073219, Längengrad: -0.1276474
Stadt: Paris, Land: FR, Breitengrad: 48.8588897, Längengrad: 2.3200410217200766
Stadt: Madrid, Land: ES, Breitengrad: 40.4167047, Längengrad: -3.7035825
Stadt: Frankfurt, Land: DE, Breitengrad: 50.1106444, Längengrad: 8.6820917
Stadt: Zurich, Land: CH, Breitengrad: 47.3744489, Längengrad: 8.5410422
Stadt: Moscow, Land: RU, Breitengrad: 55.7504461, Längengrad: 37.6174943
Stadt: Amsterdam, Land: NL, Breitengrad: 52.3727598, Längengrad: 4.8936041
Stadt: Lisbon, Land: PT, Breitengrad: 38.7077507, Längengrad: -9.1365919
Stadt: Rome, Land: IT, Breitengrad: 41.8933203, Längengrad: 12.4829321
Stadt: Dublin, Land: IE, Breitengrad: 53.3498006, Längengrad: -6.2602964
Stadt: Vienna, Land: AT, Breitengrad: 48.2083537, Längengrad: 16.3725042


In [4]:
coordinates = {}

for city in open_weather_coordinates.cities_coordinates:
    city_name = city["city_name"]
    coordinates[city_name] = {"lat":city["lat"], "lon": city["lon"]}

In [8]:
# Lade Umgebungsvariablen aus der .env-Datei (z. B. API-Schlüssel)
load_dotenv()

# API-Schlüssel aus der .env-Datei laden
api_key = os.getenv('API_KEY')

# Aktuelles Datum in Unix-Timestamp umwandeln
current_time = int(time.time())  # Aktueller Unix-Zeitstempel

# Berechne den Unix-Zeitstempel von vor 24 Stunden
start = current_time - (100*86400)  # 24 Stunden = 86400 Sekunden

In [6]:
def fetch_weather(coordinates, start, current_time, api_key):
    all_data = []  # Liste zum Speichern der Daten von allen Koordinaten
    
    for place, coord in coordinates.items():
        lat = coord['lat']
        lon = coord['lon']

        # API-Aufruf für historische Daten
        api_url = f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=day&start={start}&end={current_time}&appid={api_key}"

        try:
            # API GET-Request senden
            response = requests.get(api_url)

            if response.status_code == 200:
                data = response.json()
                print(f"API Response for Place {place} ({lat}, {lon}):", data)
                all_data.append({place: data})  # Daten zu all_data hinzufügen
            else:
                print(f"Fehlerhafte Anfrage für {place} ({lat}, {lon}). Statuscode: {response.status_code}")

        except requests.exceptions.RequestException as e:
            print(f"Ein Fehler ist aufgetreten für {place} ({lat}, {lon}): {e}")
    
    return all_data  # Gib die gesammelten Daten zurück

In [9]:
data = fetch_weather(coordinates, start, current_time, api_key)

API Response for Place London (51.5073219, -0.1276474): {'message': 'Count: 168', 'cod': '200', 'city_id': 1, 'calctime': 0.055261236, 'cnt': 168, 'list': [{'dt': 1721419200, 'main': {'temp': 298.25, 'feels_like': 298.46, 'pressure': 1013, 'humidity': 63, 'temp_min': 296.51, 'temp_max': 299.56}, 'wind': {'speed': 2.73, 'deg': 201, 'gust': 5.32}, 'clouds': {'all': 6}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}]}, {'dt': 1721422800, 'main': {'temp': 296.67, 'feels_like': 296.83, 'pressure': 1013, 'humidity': 67, 'temp_min': 294.31, 'temp_max': 298.2}, 'wind': {'speed': 2.68, 'deg': 174, 'gust': 4.47}, 'clouds': {'all': 4}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}]}, {'dt': 1721426400, 'main': {'temp': 294.76, 'feels_like': 294.91, 'pressure': 1013, 'humidity': 74, 'temp_min': 292.64, 'temp_max': 296.49}, 'wind': {'speed': 0.45, 'deg': 236, 'gust': 0.89}, 'clouds': {'all': 3}, 'weather': [{'id': 800, 'main

In [16]:
def check_entries(data):
    for city_data in data:
        city_name = list(city_data.keys())[0]  # Der Stadtnamen ist der Key des Dictionaries
        city_info = city_data[city_name]
        
        # Prüfen, ob die Liste der Einträge existiert und wie viele Einträge enthalten sind
        if "list" in city_info and len(city_info["list"]) == 100:
            print(f"{city_name} hat genau 100 Einträge.")
        else:
            print(f"{city_name} hat {len(city_info['list'])} Einträge.")

# Den Überprüfungsfunktion aufrufen
check_entries(data)

London hat 168 Einträge.
Paris hat 168 Einträge.
Madrid hat 168 Einträge.
Frankfurt hat 168 Einträge.
Zurich hat 168 Einträge.
Moscow hat 168 Einträge.
Amsterdam hat 168 Einträge.
Lisbon hat 168 Einträge.
Rome hat 168 Einträge.
Dublin hat 168 Einträge.
Vienna hat 168 Einträge.


In [13]:
df = pd.DataFrame(data)

     message  cod  city_id  calctime  cnt  \
0  Count: 48  200        1  0.005816   48   
1  Count: 48  200        1  0.005870   48   
2  Count: 48  200        1  0.005497   48   
3  Count: 48  200        1  0.184427   48   

                                                list  
0  [{'dt': 1729843200, 'main': {'temp': 286.31, '...  
1  [{'dt': 1729843200, 'main': {'temp': 283.38, '...  
2  [{'dt': 1729843200, 'main': {'temp': 287.34, '...  
3  [{'dt': 1729843200, 'main': {'temp': 283.19, '...  
